# 多类型文档处理

In [1]:
data_dir = "../notebook/C7 高级 RAG 技巧/2. 数据处理/data"

## 1. 结构化数据

In [2]:
import os
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(os.path.join(data_dir, "company.csv"))
csv_data = loader.load()

In [3]:
print(f"读取后返回的类型为：{type(csv_data)}, len={len(csv_data)}")
print(f"每个元素的类型为：{type(csv_data[0])}")
# 一行为一个page_content
print(f"第一个 page_content 中存放的数据为：\n{csv_data[0].page_content}")
print(f"第一个 metadata 中存放的数据为：{csv_data[0].metadata}")
print(f"metadata 数据类型为：{type(csv_data[0].metadata)}")

读取后返回的类型为：<class 'list'>, len=6284
每个元素的类型为：<class 'langchain_core.documents.base.Document'>
第一个 page_content 中存放的数据为：
EmployeeID: 1318
birthdate_key: 1/3/1954
age: 61
city_name: Vancouver
department: Executive
job_title: CEO
gender: M
第一个 metadata 中存放的数据为：{'source': '../notebook/C7 高级 RAG 技巧/2. 数据处理/data\\company.csv', 'row': 0}
metadata 数据类型为：<class 'dict'>


In [4]:
processed_data = csv_data[0].page_content \
                            .replace("\n", "") \
                            .replace(" ", "") \
                            .replace("Employee", "工号") \
                            .replace("birthdate_key", "的员工于") \
                            .replace("age:", "出生，现在") \
                            .replace("city_name:", "岁，居住城市为") \
                            .replace('department:', '工作部门为') \
                            .replace('job_title:', '职位是') \
                            .replace('gender:', '性别为')
processed_data = processed_data + '(M为男性F为女性)。'
print("字符串处理的结果为:", processed_data)
print('metadata中存放的路径为:', csv_data[0].metadata['source'])
print('metadata中存放的行号为:', csv_data[0].metadata['row'])

字符串处理的结果为: 工号ID:1318的员工于:1/3/1954出生，现在61岁，居住城市为Vancouver工作部门为Executive职位是CEO性别为M(M为男性F为女性)。
metadata中存放的路径为: ../notebook/C7 高级 RAG 技巧/2. 数据处理/data\company.csv
metadata中存放的行号为: 0


## 2.非结构化数据

### 2.1. PPT文件

In [5]:
from langchain.document_loaders.powerpoint import UnstructuredPowerPointLoader
loader = UnstructuredPowerPointLoader(os.path.join(data_dir, "AI视频.pptx"))
ppt_data = loader.load()

In [6]:
print(f'读取后返回的类型为:{type(ppt_data)}')
print(f'读取后返回的长度为:{len(ppt_data)}')
print(f'每个元素的类型为:{type(ppt_data[0])}')
# 非结构化的数据，所有的数据都存于page_content
print(f'第一个 page_content 中存放的数据为:\n{ppt_data[0].page_content[:200]}')
print(f'page_content 数据类型为:{type(ppt_data[0].page_content)}')
print(f'metadata 中存放的数据为:{ppt_data[0].metadata}')
print(f'metadata 数据类型为:{type(ppt_data[0].metadata)}')
print(f'metadata 存放的 source 为:', str(ppt_data[0].metadata['source']))

读取后返回的类型为:<class 'list'>
读取后返回的长度为:1
每个元素的类型为:<class 'langchain_core.documents.base.Document'>
第一个 page_content 中存放的数据为:
Sora原理与实战

动手学习AI视频



现有AI视频软件盘点

    随着2023年ChatGPT的爆发，让AI的普及率迅速提升，切实的让人感受到AI的给普通人带来的影响，在2023年火了一年的LLM后，视频领域也是也在2024年迅速崛起，前有前辈Runway，后有新生代产品pika，当然也有大名鼎鼎的开源救星SD的当家产品，Stable Video Diffusion，当这几家视频生成
page_content 数据类型为:<class 'str'>
metadata 中存放的数据为:{'source': '../notebook/C7 高级 RAG 技巧/2. 数据处理/data\\AI视频.pptx'}
metadata 数据类型为:<class 'dict'>
metadata 存放的 source 为: ../notebook/C7 高级 RAG 技巧/2. 数据处理/data\AI视频.pptx


In [7]:
# 将多个\n替换为一个\n
import re
processed_data = re.sub(r'\n+', '\n', ppt_data[0].page_content)
# 清除空格
processed_data = processed_data.replace(" ", "")
print(f"处理后的数据为:\n{processed_data[:200]}")


处理后的数据为:
Sora原理与实战
动手学习AI视频
现有AI视频软件盘点
随着2023年ChatGPT的爆发，让AI的普及率迅速提升，切实的让人感受到AI的给普通人带来的影响，在2023年火了一年的LLM后，视频领域也是也在2024年迅速崛起，前有前辈Runway，后有新生代产品pika，当然也有大名鼎鼎的开源救星SD的当家产品，StableVideoDiffusion，当这几家视频生成公司互相竞争，抢市场份


### 2.2 doc\docx文件

#### 2.2.1 一次性读取全文内容

In [8]:
from langchain_community.document_loaders.word_document import UnstructuredWordDocumentLoader

loader = UnstructuredWordDocumentLoader(os.path.join(data_dir, "1. 简介 Introduction.docx"), mode="single")
docs = loader.load()
print(f"全文内容: \n{docs[0].page_content[:200]}")


全文内容: 
第一章 简介

欢迎来到面向开发者的提示工程部分，本部分内容基于吴恩达老师的《Prompt Engineering for Developer》课程进行编写。《Prompt Engineering for Developer》课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 老师合作授课，Isa 老师曾开发过受欢迎的 ChatGPT 检索插件，并且在教授 LLM （Larg


#### 2.2.2 按元素读取内容

In [9]:
loader = UnstructuredWordDocumentLoader(os.path.join(data_dir, "1. 简介 Introduction.docx"), mode="elements")
docs = loader.load()
for i in range(len(docs)):
    print(f"第{i}个元素: \n{docs[i].page_content[:200]}", end="\n")


第0个元素: 
第一章 简介
第1个元素: 
欢迎来到面向开发者的提示工程部分，本部分内容基于吴恩达老师的《Prompt Engineering for Developer》课程进行编写。《Prompt Engineering for Developer》课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 老师合作授课，Isa 老师曾开发过受欢迎的 ChatGPT 检索插件，并且在教授 LLM （Large Langua
第2个元素: 
网络上有许多关于提示词（Prompt， 本教程中将保留该术语）设计的材料，例如《30 prompts everyone has to know》之类的文章，这些文章主要集中在 ChatGPT 的 Web 界面上，许多人在使用它执行特定的、通常是一次性的任务。但我们认为，对于开发人员，大语言模型（LLM） 的更强大功能是能通过 API 接口调用，从而快速构建软件应用程序。实际上，我们了解到 Deep
第3个元素: 
在本模块，我们将与读者分享提升大语言模型应用效果的各种技巧和最佳实践。书中内容涵盖广泛，包括软件开发提示词设计、文本总结、推理、转换、扩展以及构建聊天机器人等语言模型典型应用场景。我们衷心希望该课程能激发读者的想象力，开发出更出色的语言模型应用。
第4个元素: 
随着 LLM 的发展，其大致可以分为两种类型，后续称为基础 LLM 和指令微调（Instruction Tuned）LLM。基础LLM是基于文本训练数据，训练出预测下一个单词能力的模型。其通常通过在互联网和其他来源的大量数据上训练，来确定紧接着出现的最可能的词。例如，如果你以“从前，有一只独角兽”作为 Prompt ，基础 LLM 可能会继续预测“她与独角兽朋友共同生活在一片神奇森林中”。但是，如
第5个元素: 
与基础语言模型不同，指令微调 LLM 通过专门的训练，可以更好地理解并遵循指令。举个例子，当询问“法国的首都是什么？”时，这类模型很可能直接回答“法国的首都是巴黎”。指令微调 LLM 的训练通常基于预训练语言模型，先在大规模文本数据上进行预训练，掌握语言的基本规律。在此基础上进行进一步的训练与微调（finetune），输入是指令，输出是对这些指令的正确回复。有时还会采用RLHF（reinforce
第6个元素: 
因此，本课程将重点

## 3. 数据清洗及脱敏

### 3.1 数据清洗

In [10]:
import re

def clean_text(text: str):
    # 给出emoji范围并删除emoji
    emoji_pattern = re.compile(
        "[]"
        u"\U0001F601-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        "]+", 
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(r"", text)
    # 删除字符串开头的空格
    text = re.sub(r"^\s+", "", text)
    # 将多个回车改为一个
    text = re.sub(r"\n+", "\n", text)
    # 删除句子中（以逗号结尾）的回车
    text = re.sub(r"，\n", ",", text)

    return text

# 该字符串中存在重复的空格、换行符以及表情符号
text = '''     llm_universe是一个面向小白开发者的大模型应用开发教程😍😍😍，

旨在基于阿里云服务器😆😆😆，


结合个人知识库助手项目，
通过一个课程完成大模型开发的重点入门。


学习llm_universe之后，
我才知道大模型应用开发原来这么简单！🌟
'''

cleaned_text = clean_text(text)
print(f"清洗前的句子: \n{text}")
print(f"清洗后的句子: \n{cleaned_text}")

清洗前的句子: 
     llm_universe是一个面向小白开发者的大模型应用开发教程😍😍😍，

旨在基于阿里云服务器😆😆😆，


结合个人知识库助手项目，
通过一个课程完成大模型开发的重点入门。


学习llm_universe之后，
我才知道大模型应用开发原来这么简单！🌟

清洗后的句子: 
llm_universe是一个面向小白开发者的大模型应用开发教程,旨在基于阿里云服务器,结合个人知识库助手项目,通过一个课程完成大模型开发的重点入门。
学习llm_universe之后,我才知道大模型应用开发原来这么简单！



### 3.2 数据脱敏

In [12]:
text = """llm_universe的GitHub地址为https://github.com/datawhalechina/llm-universe
Datawhale的GitHub地址为https://github.com/datawhalechina
大模型实习生小明的邮箱为xiaoming@gmail.com"""

def remove_urls(text: str):
    r"""
    http[s]?://匹配http://或https://，其中[s]?表示可选
    ?:匹配后面的文本
    [a-zA-Z]匹配大小写字母
    [0-9]匹配数字
    [$-_@.&+!*\(\),]匹配各种符号，其中在左右括号前边加了\来转译
    |意思为或，将所有条件并列起来
    (?:%[0-9a-fA-F][0-9a-fA-F])匹配网址中URL编码比如%20，格式为%后加两位数字或字母
    """
    url_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    text = re.sub(url_pattern, "", text)

    email_pattern = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}"
    text = re.sub(email_pattern, "", text)

    return text

text = remove_urls(text)
print(text)

llm_universe的GitHub地址为
Datawhale的GitHub地址为
大模型实习生小明的邮箱为
